In [1]:
%cd /teamspace/studios/this_studio/arabic_tts/inference/arabspeak

/teamspace/studios/this_studio/arabic_tts/inference/arabspeak


In [31]:
from tts_arabic import tts
from IPython.display import Audio

In [37]:
model = text = "اَلسَّلامُ عَلَيكُم يَا صَدِيقِي. اَلسَّلامُ عَلَيكُم يَا صَدِيقِي.اَلسَّلامُ عَلَيكُم يَا صَدِيقِي."
wave = tts(text, speaker=2, pace=0.9, play=False,model_id="mixer80",vocoder_id="vocos")

In [38]:
Audio(wave,rate=22050)

In [12]:
import platform
import phonemizer
from phonemizer.backend.espeak.wrapper import EspeakWrapper
from phonemizer.separator import Separator
from phonemizer.backend import EspeakBackend

In [9]:
espeak = EspeakBackend(language="ar", with_stress=True, preserve_punctuation=True,punctuation_marks=",",)

In [24]:
espeak.phonemize(["منْ يَرضى"],separator=Separator("|"))

['mn|jˈardˤa.ː|']

In [30]:
_vowels = "iyɨʉɯuɪʏʊeøɘəɵɤoɛœɜɞʌɔæɐaɶɑɒᵻ"
_non_pulmonic_consonants = "ʘɓǀɗǃʄǂɠǁʛ"
_pulmonic_consonants = "pbtdʈɖcɟkɡqɢʔɴŋɲɳnɱmʙrʀⱱɾɽɸβfvθðszʃʒʂʐçʝxɣχʁħʕhɦɬɮʋɹɻjɰlɭʎʟ"
_suprasegmentals = "ˈˌːˑ"
_other_symbols = "ʍwɥʜʢʡɕʑɺɧʲ"
_diacrilics = "ɚ˞ɫ"
_phonemes = _vowels + _non_pulmonic_consonants + _pulmonic_consonants + _suprasegmentals + _other_symbols + _diacrilics

In [31]:
len(_phonemes)

116

In [9]:
platform.platform()

'Linux-5.15.0-1072-aws-x86_64-with-glibc2.31'

In [6]:
import os
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

In [33]:
model_path = "/teamspace/studios/this_studio/arabic_tts/training/export"
model2_path = "/teamspace/studios/this_studio/arabic_tts/training/export2"

In [34]:
model_fp32 = os.path.join(model_path,"model.onnx")
model2_fp32 = os.path.join(model2_path,"model.onnx")
model_quant = os.path.join(model_path,"model_dynamic_quant.onnx")

In [336]:
quantize_dynamic(
    model_input=model_fp32,
    model_output=model_quant,
    weight_type=QuantType.FP16 # Can be QInt8 or QUInt8
)

AttributeError: FP16

In [5]:
from IPython.display import Audio
import onnxruntime as ort
import numpy as np
import json
import re
from components.text import Characters, Tokenizer

In [9]:
with open("/teamspace/studios/this_studio/arabic_tts/training/export/tokenizer_config.json", "r") as f:
    tokenizer_config = json.load(f)

In [10]:
tokenizer = Tokenizer.init_from_config(tokenizer_config)

In [35]:
session1 = ort.InferenceSession(model_fp32)
session2 = ort.InferenceSession(model2_fp32)

In [47]:
texts = ["الشَّمْسُ تَغْرُبُ فِي الأُفُقِ، مُلَوِّنَةً السَّمَاءَ بِألْوَانٍ دَافِئَةٍ.",
         "يُرْجَى مُلَاحَظَةُ أَنَّ النَّصَّ الْمُرَادَ تَشْكِيلُهُ مَحْدُودٌ فِي عَدَدِ الأَحْرُفِ المَسْمُوحِ بِهَا. يُرْجَى مُلَاحَظَةُ أَنَّ النَّصَّ الْمُرَادَ تَشْكِيلُهُ مَحْدُودٌ فِي عَدَدِ الأَحْرُفِ المَسْمُوحِ بِهَا. ",
         "الأَدَبُ يَنْقُلُ تَجَارِبَ الشُّعُوبِ وَحِكْمَتَهُمْ عَلَى مَرِّ الْعُصُورِ.",]

In [48]:
tokenized = np.expand_dims(np.array(tokenizer.encode(texts[1])),[0])

In [49]:
scales = np.array([0.667,1,1]).astype(np.float32)

In [50]:
length = np.array([tokenized.shape[1]+10])

In [51]:
def prepare_inputs(session,inps):
    prepared = {}
    sess_inps = session.get_inputs()
    for i in range(len(sess_inps)):
        prepared[sess_inps[i].name] = inps[i]
    return prepared

In [52]:
inp = [tokenized,length,scales]

In [53]:
prepared = prepare_inputs(session, inp)

In [54]:
wav1 = session1.run(None,prepared)[0][0][0]
wav2 = session2.run(None,prepared)[0][0][0]

In [55]:
print(wav1.shape)
print(wav2.shape)

(99840,)
(193536,)


In [56]:
Audio(wav1,rate=16000)

In [57]:
Audio(wav2,rate=16000)

In [61]:
with open("/teamspace/studios/this_studio/arabic_tts/inference/arabspeak/components/metadata.txt","r",encoding="utf-16") as f:
    lines = f.readlines()
    # text = [line.strip().split("|")[1] for line in lines]
    # wav = [line.strip().split("|")[0] for line in lines]

In [59]:
whole_lines = [w+'|'+t for w,t in zip(wav,text)]

In [62]:
lines[0]

'ch_00_arabic_tts_dataset_1|.هَذَاالتَّسْجِيل .مِنْ طَرَفِ لربّري بُوكْس .جَمِيع تَسَّجِيلَات لربّري بُوكْس\n'

In [60]:
whole_lines[0]

'ch_00_arabic_tts_dataset_1|هَذَاالتَّسْجِيل .مِنْ طَرَفِ لربّري بُوكْس .جَمِيع تَسَّجِيلَات لربّري بُوكْس.\n'

In [55]:
for i in range(len(text)):
    if text[i][0] == '.':
        if text[i][-1] =='.':
            text[i] = text[i][1:-1]+'\n'
        else:
            text[i] = text[i][1:]+'.'+'\n'

In [50]:
count = 0
for t in text:
    if t[0] == '.':
        count += 1

In [5]:
new_text = [t.replace(" .", ". ") for t in text]

In [42]:
texts = []

In [43]:
for t in text:
    if(t[0] == '.'):
        if(t[-1] == ' '):
            temp = t[1:-1]+'.'
        else:
            temp = t[1:]+'.'

In [47]:
temp[120]

IndexError: string index out of range

In [8]:
wav[1000]

'ch_02_arabic_tts_dataset_433'

In [11]:
tokenizer.phonemizer.phonemize(new_text[1])[0]

'.'

In [9]:
o = tokenizer.encode(text[0])

In [10]:
tokenizer.decode(o)

'.هَذَاالتَّسْجِيل .مِنْ طَرَفِ لربّري بُوكْس .جَمِيع تَسَّجِيلَات لربّري بُوكْس'

In [9]:
re.sub(r'\s+', ' ', text)

'hello we <>want to be friends '

In [8]:
re.sub(r"[\<\>\(\)\[\]\"]+", "", text)

'hello\twe want to be friends\n'

In [4]:
phonemizer = Phonemizer.init_from_config(config=tokenizer["phonemizer"])

In [5]:
phonemizer.phonemize("أنا هنا أريد أن ألعب")

'>nA|hnA|>ryd|>n|>lEb'

In [5]:
chars = Characters.init_from_config(config=tokenizer["characters"])

In [8]:
chars.chars_to_ids(["s","b"])

[55, 38]

In [9]:
"|".join("hello bro")

'h|e|l|l|o| |b|r|o'